<a href="https://colab.research.google.com/github/hopeof-Greatmind/object_detection_ivpl/blob/master/tf_object_detection_PokerCard_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# If you forked the repository, you can replace the link.
repo_url = 'https://github.com/hopeof-Greatmind/object_detection_ivpl'

# Number of training steps.
num_steps = 1000  # 200000

# Number of evaluation steps.
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 8
    }
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'ssd_mobilenet_v2'
#selected_model = 'faster_rcnn_inception_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

In [7]:
import os

%cd /content

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

!git clone {repo_url}
%cd {repo_dir_path}
!git pull

/content
fatal: destination path 'object_detection_ivpl' already exists and is not an empty directory.
/content/object_detection_ivpl
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 8 (delta 1), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.
From https://github.com/hopeof-Greatmind/object_detection_ivpl
   ad60785..887c223  master     -> origin/master
Updating ad60785..887c223
Fast-forward
 export_inference_graph.py                        |  150 ++
 local_inference_test.ipynb                       |  465 ++++
 local_inference_test.py                          |  156 ++
 tensorflow_object_detection_training_colab.ipynb | 2757 ++++++++++++++++++++++
 4 files changed, 3528 insertions(+)
 create mode 100644 export_inference_graph.py
 create mode 100644 local_inference_test.ipynb
 create mode 100644 local_inference_test.py
 create mode 100644 tensorflow_object_detection_trai

In [8]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!python object_detection/builders/model_builder_test.py

/content
fatal: destination path 'models' already exists and is not an empty directory.
/content/models/research
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTest.test_create_experimental_model
[       OK ] ModelBuilderTest.test_create_experimental_model
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[       OK ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[       OK ] ModelBuilderTest.test_create_faster_rcnn_models_from_con

In [9]:
%cd {repo_dir_path}

# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python xml_to_csv.py -i data/images/train -o data/annotations/train_labels.csv -l data/annotations

# Convert test folder annotation xml files to a single csv.
!python xml_to_csv.py -i data/images/test -o data/annotations/test_labels.csv

# Generate `train.record`
!python generate_tfrecord.py --csv_input=data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=data/images/train --label_map data/annotations/label_map.pbtxt

# Generate `test.record`
!python generate_tfrecord.py --csv_input=data/annotations/test_labels.csv --output_path=data/annotations/test.record --img_path=data/images/test --label_map data/annotations/label_map.pbtxt

/content/object_detection_ivpl
Traceback (most recent call last):
  File "xml_to_csv.py", line 36, in <module>
    main()
  File "xml_to_csv.py", line 32, in main
    xml_df.to_csv(('images/' + folder + '_labels.csv'), index=None)
  File "/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py", line 3228, in to_csv
    formatter.save()
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/formats/csvs.py", line 183, in save
    compression=self.compression,
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/common.py", line 399, in _get_handle
    f = open(path_or_buf, mode, encoding=encoding, newline="")
FileNotFoundError: [Errno 2] No such file or directory: 'images/train_labels.csv'
Traceback (most recent call last):
  File "xml_to_csv.py", line 36, in <module>
    main()
  File "xml_to_csv.py", line 32, in main
    xml_df.to_csv(('images/' + folder + '_labels.csv'), index=None)
  File "/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py", line 3228, in to_